In [3]:
from pyspark.sql import SparkSession

ss = SparkSession.builder.master('local').appName('newbe spark-sql').getOrCreate()

In [55]:
from pyspark.sql.types import StringType, IntegerType, FloatType, StructType, StructField

In [56]:
stocks = [
    ('Google', 'GOOGL', 'USA', 2984, 'USD'),
    ('Netflix', 'NFLX', 'USA', 645, 'USD'),
    ('Amazon', 'AMZN', 'USA', 3518, 'USD'),
    ('Tesla', 'TSLA', 'USD', 1222, 'USD'),
    ('Tencent', '0070', 'HONG KONG', 483, 'HKD'),
    ('Toyota', '7203', 'Japan', 2006, 'JPY'),
    ('Samsung', '005930', 'Korea', 70600, 'KRW'),
    ('Kakao', '035720', 'Korea', 125000, 'KRW'),
]

stockschema = ['name', 'ticker', 'country', 'price', 'currency']

earnings = [
    ('Google', 27.99, 'USD'),
    ('Netflix', 2.56, 'USD'),
    ('Amazon', 6.12, 'USD'),
    ('Tesla', 1.86, 'USD'),
    ('Tencent', 11.01, 'HKD'),
    ('Toyota', 224.82, 'JPY'),
    ('Samsung', 1780., 'KRW'),
    ('Kakao', 705., 'KRW'),    
]

earningschema = StructType([
    StructField('name', StringType(), True),
    StructField('eps', FloatType(), True),
    StructField('currency', StringType(), True)
])

In [9]:
df = ss.createDataFrame(data = stocks, schema = stockschema)

In [57]:
df2 = ss.createDataFrame(data = earnings, schema = earningschema)

In [12]:
df.dtypes

[('name', 'string'),
 ('ticker', 'string'),
 ('country', 'string'),
 ('price', 'bigint'),
 ('currency', 'string')]

In [58]:
df2.dtypes

[('name', 'string'), ('eps', 'float'), ('currency', 'string')]

In [10]:
df.show()

+-------+------+---------+------+--------+
|   name|ticker|  country| price|currency|
+-------+------+---------+------+--------+
| Google| GOOGL|      USA|  2984|     USD|
|Netflix|  NFLX|      USA|   645|     USD|
| Amazon|  AMZN|      USA|  3518|     USD|
|  Tesla|  TSLA|      USD|  1222|     USD|
|Tencent|  0070|HONG KONG|   483|     HKD|
| Toyota|  7203|    Japan|  2006|     JPY|
|Samsung|005930|    Korea| 70600|     KRW|
|  Kakao|035720|    Korea|125000|     KRW|
+-------+------+---------+------+--------+



In [59]:
df2.show()

+-------+------+--------+
|   name|   eps|currency|
+-------+------+--------+
| Google| 27.99|     USD|
|Netflix|  2.56|     USD|
| Amazon|  6.12|     USD|
|  Tesla|  1.86|     USD|
|Tencent| 11.01|     HKD|
| Toyota|224.82|     JPY|
|Samsung|1780.0|     KRW|
|  Kakao| 705.0|     KRW|
+-------+------+--------+



In [13]:
# if you want to use dateframe in pyspark-sql, have to enroll at temporary view.

df.createOrReplaceTempView('stocks')

In [60]:
df2.createOrReplaceTempView('earnings')

In [25]:
ss.sql("select name, price from stocks where price >= 2000 and currency != 'JPY'").show()

+-------+------+
|   name| price|
+-------+------+
| Google|  2984|
| Amazon|  3518|
|Samsung| 70600|
|  Kakao|125000|
+-------+------+



In [41]:
ss.sql("select name, price, currency from stocks \
       where currency = 'USD' and\
       price > (select price from stocks where ticker = 'NFLX')\
       order by price desc").show()

+------+-----+--------+
|  name|price|currency|
+------+-----+--------+
|Amazon| 3518|     USD|
|Google| 2984|     USD|
| Tesla| 1222|     USD|
+------+-----+--------+



In [40]:
ss.sql("select name, price, currency from stocks \
       where currency = 'USD' and\
       price > (select price from stocks where ticker = 'NFLX')\
       order by length(name) asc").show()

+------+-----+--------+
|  name|price|currency|
+------+-----+--------+
| Tesla| 1222|     USD|
|Google| 2984|     USD|
|Amazon| 3518|     USD|
+------+-----+--------+



In [49]:
ss.sql("select * from stocks").show()

+-------+------+---------+------+--------+
|   name|ticker|  country| price|currency|
+-------+------+---------+------+--------+
| Google| GOOGL|      USA|  2984|     USD|
|Netflix|  NFLX|      USA|   645|     USD|
| Amazon|  AMZN|      USA|  3518|     USD|
|  Tesla|  TSLA|      USD|  1222|     USD|
|Tencent|  0070|HONG KONG|   483|     HKD|
| Toyota|  7203|    Japan|  2006|     JPY|
|Samsung|005930|    Korea| 70600|     KRW|
|  Kakao|035720|    Korea|125000|     KRW|
+-------+------+---------+------+--------+



In [52]:
# when you use case,  attach ',' behind select part like this 'select *,' 

ss.sql("select *, \
       case when price >= 2000 then 'alert'\
       else 'None'\
       end as caution\
       from stocks").show()

+-------+------+---------+------+--------+-------+
|   name|ticker|  country| price|currency|caution|
+-------+------+---------+------+--------+-------+
| Google| GOOGL|      USA|  2984|     USD|  alert|
|Netflix|  NFLX|      USA|   645|     USD|   None|
| Amazon|  AMZN|      USA|  3518|     USD|  alert|
|  Tesla|  TSLA|      USD|  1222|     USD|   None|
|Tencent|  0070|HONG KONG|   483|     HKD|   None|
| Toyota|  7203|    Japan|  2006|     JPY|  alert|
|Samsung|005930|    Korea| 70600|     KRW|  alert|
|  Kakao|035720|    Korea|125000|     KRW|  alert|
+-------+------+---------+------+--------+-------+



In [53]:
ss.sql("select *, \
       if (stocks.price >= 2000, 'alert', 'None') as caution\
       from stocks").show()

+-------+------+---------+------+--------+-------+
|   name|ticker|  country| price|currency|caution|
+-------+------+---------+------+--------+-------+
| Google| GOOGL|      USA|  2984|     USD|  alert|
|Netflix|  NFLX|      USA|   645|     USD|   None|
| Amazon|  AMZN|      USA|  3518|     USD|  alert|
|  Tesla|  TSLA|      USD|  1222|     USD|   None|
|Tencent|  0070|HONG KONG|   483|     HKD|   None|
| Toyota|  7203|    Japan|  2006|     JPY|  alert|
|Samsung|005930|    Korea| 70600|     KRW|  alert|
|  Kakao|035720|    Korea|125000|     KRW|  alert|
+-------+------+---------+------+--------+-------+



In [61]:
df2.select('*').show()

+-------+------+--------+
|   name|   eps|currency|
+-------+------+--------+
| Google| 27.99|     USD|
|Netflix|  2.56|     USD|
| Amazon|  6.12|     USD|
|  Tesla|  1.86|     USD|
|Tencent| 11.01|     HKD|
| Toyota|224.82|     JPY|
|Samsung|1780.0|     KRW|
|  Kakao| 705.0|     KRW|
+-------+------+--------+



In [69]:
ss.sql("select stocks.*, earnings.eps from stocks \
       inner join earnings on stocks.name = earnings.name").show()

+-------+------+---------+------+--------+------+
|   name|ticker|  country| price|currency|   eps|
+-------+------+---------+------+--------+------+
| Amazon|  AMZN|      USA|  3518|     USD|  6.12|
| Google| GOOGL|      USA|  2984|     USD| 27.99|
|  Kakao|035720|    Korea|125000|     KRW| 705.0|
|Netflix|  NFLX|      USA|   645|     USD|  2.56|
|Samsung|005930|    Korea| 70600|     KRW|1780.0|
|Tencent|  0070|HONG KONG|   483|     HKD| 11.01|
|  Tesla|  TSLA|      USD|  1222|     USD|  1.86|
| Toyota|  7203|    Japan|  2006|     JPY|224.82|
+-------+------+---------+------+--------+------+



In [73]:
ss.sql("select stocks.*, earnings.eps,\
       round(stocks.price/earnings.eps, 2) as per \
       from stocks \
       join earnings on stocks.name = earnings.name").show()

+-------+------+---------+------+--------+------+------+
|   name|ticker|  country| price|currency|   eps|   per|
+-------+------+---------+------+--------+------+------+
| Amazon|  AMZN|      USA|  3518|     USD|  6.12|574.84|
| Google| GOOGL|      USA|  2984|     USD| 27.99|106.61|
|  Kakao|035720|    Korea|125000|     KRW| 705.0| 177.3|
|Netflix|  NFLX|      USA|   645|     USD|  2.56|251.95|
|Samsung|005930|    Korea| 70600|     KRW|1780.0| 39.66|
|Tencent|  0070|HONG KONG|   483|     HKD| 11.01| 43.87|
|  Tesla|  TSLA|      USD|  1222|     USD|  1.86|656.99|
| Toyota|  7203|    Japan|  2006|     JPY|224.82|  8.92|
+-------+------+---------+------+--------+------+------+

